In [4]:
from herbie import Herbie, FastHerbie
import herbie
#from toolbox import EasyMap, pc
#from paint.standard2 import cm_tmp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import shapely.geometry as sgeom
import cartopy.feature
import cartopy.feature as cfeature
from cartopy.feature import ShapelyFeature
from cartopy.io.shapereader import Reader
import numpy as np
#from toolbox.cartopy_tools import common_features, pc
import pandas as pd
import numpy as np
import glob
import matplotlib.cm as cm
import xarray as xr
from datetime import datetime, timedelta
import time
import os
import warnings
import pickle
warnings.filterwarnings("ignore", message="Will not remove GRIB file because it previously existed.")
warnings.filterwarnings("ignore", category=FutureWarning, message=".*decode_timedelta.*")



"""

HRRR model documentation:
https://rapidrefresh.noaa.gov/hrrr/

Documentation for the HRRR data download wrapper:
https://herbie.readthedocs.io/en/stable/

Variables:
https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml


"""

'\n\nHRRR model documentation:\nhttps://rapidrefresh.noaa.gov/hrrr/\n\nDocumentation for the HRRR data download wrapper:\nhttps://herbie.readthedocs.io/en/stable/\n\nVariables:\nhttps://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfnatf00.grib2.shtml\n\n\n'

In [51]:
#%% Define

# date range with one-day frequency
date_range = pd.date_range(datetime(2020, 9, 5), 
                           datetime(2020, 9, 6), 
                           freq="h").tolist()#[::-1]   # list starts from end

# Forecast hour (0=analysis, 2 is recommended)
fxx = 0  




In [52]:
H = Herbie(date_range[1], model='hrrr', product='nat', fxx=fxx, verbose=False)

hrrr = H.xarray()


print (H)

/home/uegerer/.conda-envs/thrive/lib/python3.12/site-packages/herbie/core.py:1117: UserWarning: Will not remove GRIB file because Herbie will only remove subsetted files (not full files).
  warnings.warn(


║HERBIE╠ HRRR:nat


In [53]:
hrrr[0]

<xarray.Dataset>
Dimensions:                (y: 1059, x: 1799)
Coordinates:
    time                   datetime64[ns] 2020-09-05T01:00:00
    step                   timedelta64[ns] 00:00:00
    adiabaticCondensation  float64 0.0
    latitude               (y, x) float64 ...
    longitude              (y, x) float64 ...
    valid_time             datetime64[ns] ...
    gribfile_projection    object None
Dimensions without coordinates: y, x
Data variables:
    gh                     (y, x) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 nat
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             /home/uegerer/data/hrrr/20200905/hrrr.t01z.wrfna...
    local_grib:              /home/uegerer/data/hrrr/20200905/hrrr.t01z.wrfna...
    search:                  None

In [65]:
# Subhourly data
H = Herbie(date_range[1], model='hrrr', product='subh', fxx=fxx, verbose=False)
df = H.inventory()[["variable", "search_this"]]
filtered_df = df[df["search_this"].str.contains(r"\b(?:ugrd|vgrd)\b", case=False, na=False) & 
                 df["search_this"].str.contains("ave", case=False, na=False)]
wind_search_string_subh = "|".join(filtered_df["search_this"].drop_duplicates().values)

print(wind_search_string_subh)

:UGRD:10 m above ground:0-0 day ave fcst|:VGRD:10 m above ground:0-0 day ave fcst


In [66]:
H.xarray(wind_search_string_subh)

<xarray.Dataset>
Dimensions:              (y: 1059, x: 1799)
Coordinates:
    time                 datetime64[ns] 2020-09-05T01:00:00
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 10.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           datetime64[ns] 2020-09-05T01:00:00
    gribfile_projection  object None
Dimensions without coordinates: y, x
Data variables:
    u10                  (y, x) float32 -5.06 -5.04 -5.01 ... 0.42 0.26 0.11
    v10                  (y, x) float32 -3.63 -3.72 -3.73 ... 4.67 4.87 5.08
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 subh
    description:             High-Resolution Rapid Refresh - CONUS
    remote_grib:             https://noaa-hrrr-bdp-pds.s3.amazonaws.com/hrrr....
    local_grib:              /home/uegerer/data/hrrr/20200905/subset_52efb06e...
    search:                  :UGRD:10 m above ground:0-0 day ave fcst|:VGRD:1...

In [87]:
# Surface data
H = Herbie(date_range[2], model='hrrr', product='sfc', fxx=fxx, verbose=False)

                # safe_xarray(H_sfc, wind_search_string_sfc),
                # safe_xarray(H_sfc, ':TMP:2 m'),
                # safe_xarray(H_sfc, 'GUST|:PRES:surface:2 hour fcst|:SNOWC:surface:2 hour fcst|:APCP:surface:0-2 hour acc fcst|:SFCR:surface:2 hour fcst|:VEG:surface:2 hour fcst|:VGTYP:surface:2 hour fcst|:HPBL:surface:2 hour fcst'),
                # safe_xarray(H_sfc, '[U|V]GRD:10 m'),
                # safe_xarray(H_sfc, '[U|V]GRD:80 m').rename({"u": "u80", "v": "v80"}),
                # safe_xarray(H_sfc, ':DPT:2 m above ground:2 hour fcst'),
                # safe_xarray(H_sfc, ':TCDC:boundary layer cloud layer:2 hour fcst'),
                # safe_xarray(H_sfc, ':CAPE:0-3000 m above ground:2 hour fcst')

search_string = ':HPBL:surface:2 hour fcst'

H.xarray(search_string)

FileNotFoundError: [Errno 2] No such file or directory: '/home/uegerer/data/hrrr/20200905/subset_f9efb1fe__hrrr.t02z.wrfsfcf00.grib2'